## Блок 2

### Шаг 2

In [0]:
import pandas as pd
data = pd.read_csv('keywords.csv')
data['daily_shows'] = data['shows'] / 11

# Посчитаем столбец ratio и его среднее значение
data['ratio'] = data['daily_shows'] / data['shows']
data['ratio'].mean()

0.09090909090909087

## Блок 3

### Шаг 1

In [0]:
data = data[data.keyword.str.contains('недвиж')].reset_index()
len(data)

142

### Шаг 2

In [0]:
# Узнаем тип переменных из нового столбца
data['split'] = data['keyword'].str.split(' ')
type(data.loc[0]['split'])

list

### Шаг 4

In [0]:
# Переведем столбец к типу int
data['daily_shows'] = data['daily_shows'].astype(int)
data[data['keyword'] == 'недвижимость']['daily_shows']

2    6634
Name: daily_shows, dtype: int64

### Шаг 5

In [0]:
# Прибавим столбец, где будет храниться количество слов в столбце split
data['count_words'] = data['split'].apply(lambda x: len(x))

# Найдем наиболее встречающееся число слов
data['count_words'].value_counts()

3    72
2    25
4    20
5    12
6     7
8     3
9     1
7     1
1     1
Name: count_words, dtype: int64

## Блок 4

### Шаги 1, 2, 3, 4

In [0]:
# Выполняем предварительные действия, описанные в модуле
import pandas as pd
ad_stats = pd.read_excel('ad_campaigns.xlsx')
ad_stats.rename(columns={'Название группы': 'group', 'Фраза (с минус-словами)': 'phrase', 

                         'Продуктивность': 'effect', 'ID объявления': 'ad_id', 'Заголовок': 'title', 

                         'Текст': 'text', 'Ссылка': 'link'}, inplace=True)
url = ad_stats.loc[0, 'link']
from urllib import parse
parsed = parse.urlsplit(url)
print(parsed)
print(parsed.netloc)
print(parsed.query) # ответ на вопрос

SplitResult(scheme='https', netloc='awesome-site.ru', path='/', query='utm_source=yandex&utm_medium=cpc&utm_campaign=a825749b87&utm_content=dev_{device_type}', fragment='')
awesome-site.ru
utm_source=yandex&utm_medium=cpc&utm_campaign=a825749b87&utm_content=dev_{device_type}


### Шаг 6

In [0]:
# Напишем функцию 
def campaign_name(row):

    """Получение названия кампании из ссылки внутри строки row"""

    parsed = parse.urlsplit(row['link'])

    params_dict = parse.parse_qs(parsed.query)

    return params_dict['utm_campaign'][0]

# Применим функцию
ad_stats['campaign'] = ad_stats.apply(campaign_name, axis=1)

# Считаем среднее число в названии компании
ad_stats['count'] = ad_stats['campaign'].apply(lambda x: len(x))
print(ad_stats['count'].mean())

10.0


## Блок 5

### Шаг 4

In [0]:
# Построим рейтинг компаний, начиная с самого малого значения стобца effect
ad_stats.groupby('campaign').min().sort_values('effect').head()

,group,phrase,effect,ad_id,title,text,link,count
campaign,,,,,,,,
8e77a0e565,1 5 москва мрт тесла,"""!1 !мрт !нагатинский""",1.6,2101705401,МРТ центры в Москве. От 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,10
7f35591a28,1 градская мрт,"""!1 !градская !мрт""",1.7,2101705009,МРТ в Москве от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,10
be97523396,мрт в бабушкинском районе,"""!мрт !в !москве !метро !щелковская""",1.8,2101705493,МРТ у метро в Москве от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,10
d96029561c,круглосуточный мрт в москве,"""!круглосуточный !мрт !в !москве""",1.8,2428555742,МРТ круглосуточно. От 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,10
acc3077e55,алтуфьево бибирево мрт,"""!алтуфьево !бибирево !мрт""",1.9,2428554791,Томография в Москве. От 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,10


### Шаг 8

In [0]:
# Найдем компанию с минимальной суммой столбца effect
ad_stats.groupby('campaign').sum().sort_values('effect').head()

,effect,ad_id,count
campaign,,,
63153306ff,2.8,2428554756,10
f80e902d76,3.3,2101705003,10
5a7508fa8d,3.5,2428554764,10
d5b16c7769,3.6,3401169381,10
7b17b2d188,5.8,2101705006,10


## Блок 6

In [0]:
data = pd.read_csv('keywords.csv')

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

# Напишем функцию, которая по городу определяет тип региона
def geo_classification(row):
    
    # Пробегаемся по всем городам всех регионов и,
    # если такой город имеется в запросе, то возвращаем регион
    for region, city_list in geo_data.items():
        for city in city_list:
            if city in row['keyword']:
                return region
    # Если такого города нет, то возвращаем 'undefined'
    return 'undefined'

# Применяем
data['city_group'] = data.apply(geo_classification, axis=1)

data.groupby('city_group').count().head()

,keyword,shows
city_group,,
undefined,99253,99253
Дальний Восток,94,94
Северо-Запад,268,268
Центр,385,385
